In [1]:
import numpy as np

In [2]:
def prepare_data(filename: str) -> np.ndarray:
    data = np.fromfile(filename, dtype='float32')
    data = data.reshape(-1, 17)
    num_seq = data.shape[0] - (data.shape[0] % 8)
    data = data[:num_seq]
    data = data.reshape(-1, 8, 17)
    return data

In [3]:
both = prepare_data('./both_eyes.bin')
left = prepare_data('./left.bin')
right = prepare_data('./right.bin')

In [7]:
num_both = both.shape[0]
num_left = left.shape[0]
num_right = right.shape[0]

In [8]:
num_all = num_both + num_left + num_right

In [9]:
labels = np.zeros(shape=(num_all, 1))

In [10]:
labels[:num_both] = 0
labels[num_both:num_both+num_left] = 1
labels[num_both+num_left:] = 2

In [12]:
from collections import Counter

In [14]:
Counter(labels.flatten())

Counter({0.0: 1532, 1.0: 789, 2.0: 917})

In [17]:
all_data = np.concatenate((both, left, right))

In [18]:
all_data.shape

(3238, 8, 17)

In [26]:
mean_all = all_data.mean(axis=1)

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
mean_all.shape, labels.shape

((3238, 17), (3238, 1))

In [29]:
x_tr, x_te, y_tr, y_te = train_test_split(mean_all, labels)

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
lr = LogisticRegression()

In [32]:
lr.fit(x_tr, y_tr)

/Users/Marcin/.pyenv/versions/3.6.1/envs/MachineLearning/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [33]:
predictions = lr.predict(x_te)

In [35]:
from sklearn.metrics import classification_report

In [36]:
print(classification_report(y_te, predictions))

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00       369
        1.0       1.00      1.00      1.00       210
        2.0       1.00      1.00      1.00       231

avg / total       1.00      1.00      1.00       810



In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
rf = RandomForestClassifier()

In [43]:
rf.fit(x_tr, y_tr)

/Users/Marcin/.pyenv/versions/3.6.1/envs/MachineLearning/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [44]:
pred_rf = rf.predict(x_te)

In [45]:
print(classification_report(y_te, pred_rf))

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00       369
        1.0       1.00      1.00      1.00       210
        2.0       1.00      1.00      1.00       231

avg / total       1.00      1.00      1.00       810



In [49]:
channels = list(range(1, 17))

In [50]:
dict(zip(channels, rf.feature_importances_))

{1: 0.08664111450758545,
 2: 0.0006236325043264895,
 3: 0.07198292524013178,
 4: 0.0019564546323258135,
 5: 0.0010144163362607182,
 6: 0.11671980003698985,
 7: 0.05317307152532748,
 8: 0.31663591361473953,
 9: 0.06748191110898019,
 10: 0.14856262760703473,
 11: 0.1319664710316471,
 12: 0.00010384994367487867,
 13: 0.0003092183074480371,
 14: 0.0,
 15: 0.0005104488756900819,
 16: 0.0012787271956503127}